---
<center>
<h1>  alx-low_level_programming </h1>
</center>

---


In [1]:
%%bash
cd /home/elkomy/alx/simple_shell//////

In [3]:
import os
# creating 0x02-shell_redirectionss

folder_path = r'/home/elkomy/alx/simple_shell/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Create a README.md inside alx-low_level_programming/0x0D-preprocessor
file_path = os.path.join(folder_path, 'README.md')

with open(file_path, 'w') as file:
    file.write("This is a README.md file inside alx-low_level_programming/0x0D-preprocessor")



In [4]:
%%file /home/elkomy/alx/simple_shell//////README.md
This is a README.md

Overwriting /home/elkomy/alx/simple_shell/README.md


###
---
<center>
<h1> 0x16. C - Simple Shell </h1>
</center>

---


###
---
<center>
<h4> 0. Object-like Macro</h4>
</center>

---


#### Create a header file that defines a macro named `SIZE` as an abbreviation for the token `1024`.


```bash
julien@ubuntu:~/0x0c. macro, structures$ cat 0-main.c
#include "0-object_like_macro.h"
#include "0-object_like_macro.h"
#include <stdio.h>

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = 98 + SIZE;
    printf("%d\n", s);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 0-main.c -o a
julien@ubuntu:~/0x0c. macro, structures$ ./a 
1122
julien@ubuntu:~/0x0c. macro, structures$ 

```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```0-object_like_macro.h```

In [18]:
%%file /home/elkomy/alx/simple_shell/0-main.c
#include <stdio.h>

/**
* main - Entry point
*
* Return: Always 0 (Success)
*/
int main(void)
{
printf("Betty would be proud!\n");
return (0);
}


Overwriting /home/elkomy/alx/simple_shell/0-main.c


In [34]:
%%bash
betty 0-main.c



========== 0-main.c ==========
main


In [19]:
!pwd

/home/elkomy/alx/simple_shell


In [36]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 0-main.c -o a.out
./a.out


Betty would be proud!


In [37]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main add5e92] alx-low_level_programming/alx/simple_shell0-putchar
 4 files changed, 892 insertions(+), 1 deletion(-)
 create mode 100644 0-main.c
 create mode 100644 0x16. C - Simple Shell.ipynb
 create mode 100755 a.out


To https://github.com/elkomysara/simple_shell.git
   5e061ea..add5e92  main -> main


###
---
<center>
<h4>1. Simple shell 0.1</h4>
</center>

---


#### Write a UNIX command line interpreter.

* Usage: `simple_shell`
#### Your Shell should:

* Display a prompt and wait for the user to type a command. A command line always ends with a new line.
* The prompt is displayed again each time a command has been executed.
* The command lines are simple, no semicolons, no pipes, no redirections or any other advanced features.
* The command lines are made only of one word. No arguments will be passed to programs.
* If an executable cannot be found, print an error message and display the prompt again.
* Handle errors.
* You have to handle the “end of file” condition (`Ctrl+D`)
#### You don’t have to:

* use the `PATH`
* implement built-ins
* handle special characters : `", ', ```, \, *, &, #`
* be able to move the cursor
* handle commands with arguments
`execve` will be the core part of your Shell, don’t forget to pass the environ to it…

```bash
julien@ubuntu:~/shell$ ./shell 
#cisfun$ ls
./shell: No such file or directory
#cisfun$ /bin/ls
barbie_j       env-main.c  exec.c  fork.c  pid.c  ppid.c    prompt   prompt.c  shell.c  stat.c         wait
env-environ.c  exec    fork    mypid   ppid   printenv  promptc  shell     stat test_scripting.sh  wait.c
#cisfun$ /bin/ls -l
./shell: No such file or directory
#cisfun$ ^[[D^[[D^[[D
./shell: No such file or directory
#cisfun$ ^[[C^[[C^[[C^[[C
./shell: No such file or directory
#cisfun$ exit
./shell: No such file or directory
#cisfun$ ^C
julien@ubuntu:~/shell$ echo "/bin/ls" | ./shell
barbie_j       env-main.c  exec.c  fork.c  pid.c  ppid.c    prompt   prompt.c  shell.c  stat.c         wait
env-environ.c  exec    fork    mypid   ppid   printenv  promptc  shell     stat test_scripting.sh  wait.c
#cisfun$ julien@ubuntu:~/shell$

```
#### Repo:
* GitHub repository: ``` simple_shell ```

In [8]:
%%file alx/simple_shell1-main.c
#include "1-pi.h"
#include "1-pi.h"
#include <stdio.h>

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    float a;
    float r;

    r = 98;
    a = PI * r * r;
    printf("%.3f\n", a);
    return (0);
}

Writing 0x0D-preprocessor/1-main.c


In [9]:
%%file alx/simple_shell1-pi.h
#ifndef PI_H
#define PI_H

#define PI 3.14159265359

#endif /* PI_H */


Writing 0x0D-preprocessor/1-pi.h


In [10]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell1-main.c -o alx/simple_shellb
./alx/simple_shellb


30171.855


In [11]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main dbdf94f] alx-low_level_programming/0x0D-preprocessor/0-putchar
 3 files changed, 25 insertions(+)
 create mode 100644 0x0D-preprocessor/1-main.c
 create mode 100644 0x0D-preprocessor/1-pi.h
 create mode 100755 0x0D-preprocessor/b


To https://github.com/elkomysara/alx-low_level_programming.git
   2a7b388..dbdf94f  main -> main


###
---
<center>
<h4> 2. File name</h4>
</center>

---


#### Write a program that prints the name of the file it was compiled from, followed by a new line.

* You are allowed to use the standard library

```bash
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 2-main.c -o c
julien@ubuntu:~/0x0c. macro, structures$ ./c 
2-main.c
julien@ubuntu:~/0x0c. macro, structures$ cp 2-main.c 02-main.c
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 02-main.c -o cc
julien@ubuntu:~/0x0c. macro, structures$ ./cc
02-main.c
julien@ubuntu:~/0x0c. macro, structures$   
```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```2-main.c```

In [13]:
%%file alx/simple_shell2-main.c
#include <stdio.h>

/**
* main - prints the name of the file it was compiled from
*
* Return: Always 0.
*/
int main(void)
{
printf("%s\n", __FILE__);
return (0);
}


Overwriting 0x0D-preprocessor/2-main.c


In [15]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell2-main.c -o alx/simple_shellc
./alx/simple_shellc
cp alx/simple_shell2-main.c alx/simple_shell02-main.c
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell02-main.c -o cc


0x0D-preprocessor/2-main.c


In [16]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main d83673d] alx-low_level_programming/0x0D-preprocessor/0-putchar
 4 files changed, 24 insertions(+)
 create mode 100644 0x0D-preprocessor/02-main.c
 create mode 100644 0x0D-preprocessor/2-main.c
 create mode 100755 0x0D-preprocessor/c
 create mode 100755 cc


To https://github.com/elkomysara/alx-low_level_programming.git
   dbdf94f..d83673d  main -> main


###
---
<center>
<h4>3. Function-like macro</h4>
</center>

---


#### Write a function-like macro `ABS(x)` that computes the absolute value of a number `x`.

```bash
julien@ubuntu:~/0x0c. macro, structures$ cat 3-main.c
#include <stdio.h>
#include "3-function_like_macro.h"
#include "3-function_like_macro.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int i;
    int j;

    i = ABS(-98) * 10;
    j = ABS(98) * 10;
    printf("%d, %d\n", i, j);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 3-main.c -o d
julien@ubuntu:~/0x0c. macro, structures$ ./d 
980, 980
julien@ubuntu:~/0x0c. macro, structures$ 

```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```3-function_like_macro.h```

In [18]:
%%file alx/simple_shell3-main.c
#include <stdio.h>
#include "3-function_like_macro.h"
#include "3-function_like_macro.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int i;
    int j;

    i = ABS(-98) * 10;
    j = ABS(98) * 10;
    printf("%d, %d\n", i, j);
    return (0);
}

Overwriting 0x0D-preprocessor/3-main.c


In [19]:
%%file alx/simple_shell3-function_like_macro.h
#ifndef FUNCTION_LIKE_MACRO_H
#define FUNCTION_LIKE_MACRO_H

#define ABS(x) ((x) < 0 ? -(x) : (x))

#endif /* FUNCTION_LIKE_MACRO_H */


Writing 0x0D-preprocessor/3-function_like_macro.h


In [21]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell3-main.c -o alx/simple_shelld
./alx/simple_shelld


980, 980


In [22]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main 4e412ae] alx-low_level_programming/0x0D-preprocessor/0-putchar
 3 files changed, 25 insertions(+)
 create mode 100644 0x0D-preprocessor/3-function_like_macro.h
 create mode 100644 0x0D-preprocessor/3-main.c
 create mode 100755 0x0D-preprocessor/d


To https://github.com/elkomysara/alx-low_level_programming.git
   d83673d..4e412ae  main -> main


###
---
<center>
<h4>4. SUM</h4>
</center>

---


#### Write a function-like macro `SUM(x, y)` that computes the sum of the numbers `x` and `y`.

```bash
 julien@ubuntu:~/0x0c. macro, structures$ cat 4-main.c
#include <stdio.h>
#include "4-sum.h"
#include "4-sum.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = SUM(98, 1024);
    printf("%d\n", s);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 4-main.c -o e
julien@ubuntu:~/0x0c. macro, structures$ ./e 
1122
julien@ubuntu:~/0x0c. macro, structures$ 
```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```4-sim.h```

In [23]:
%%file alx/simple_shell4-sum.h

#ifndef SUM_H
#define SUM_H

#define SUM(x, y) ((x) + (y))

#endif /* SUM_H */


Writing 0x0D-preprocessor/4-sum.h


In [24]:
%%file alx/simple_shell4-main.c

#include <stdio.h>
#include "4-sum.h"
#include "4-sum.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = SUM(98, 1024);
    printf("%d\n", s);
    return (0);
}



Writing 0x0D-preprocessor/4-main.c


In [25]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell4-main.c -o alx/simple_shelle
./alx/simple_shelle


1122


In [26]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main ca563fe] alx-low_level_programming/0x0D-preprocessor/0-putchar
 3 files changed, 26 insertions(+)
 create mode 100644 0x0D-preprocessor/4-main.c
 create mode 100644 0x0D-preprocessor/4-sum.h
 create mode 100755 0x0D-preprocessor/e


To https://github.com/elkomysara/alx-low_level_programming.git
   4e412ae..ca563fe  main -> main
